In [1]:
# https://www.kaggle.com/aherparesh/sentiment-analysis-on-amazon-product-rnn-97-acc
# https://www.kaggle.com/lele1995/amazon-reviews-sentiment-analysis

In [3]:
import pandas as pd
import numpy as np
import pickle
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

test_data=pd.read_csv('test_data_sentiment_analysis.csv').drop('Unnamed: 0',axis=1)
test_data

# load the model from disk
trained_model_rf= pickle.load(open('trained_model_rf_sentiment_analysis_10872_features.pkl', 'rb'))
print(trained_model_rf)

# Testing phase
trained_tfidf_vector= pickle.load(open("tfidf1.pkl", 'rb'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vimald\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vimald\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vimald\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,reviews.text,reviews.rating
0,AVpjEN4jLJeJML43rpUe,I can't update Facebook and a few other apps. ...,2.0
1,AV1YnRtnglJLPUi8IJmV,I bought this tablet for my boyfriend and he a...,5.0
2,AVphgVaX1cnluZ0-DR74,Bought a gift for my mom. She absolutely loved...,5.0
3,AVqkIhwDv8e3D1O-lebb,Love the kindle and the price was excellent. I...,5.0
4,AVpfpK8KLJeJML43BCuD,The Amazon Tap has great sound and is responsi...,5.0
...,...,...,...
7722,AV1YnRtnglJLPUi8IJmV,First purchase of an e reader. Kindle Paperwhi...,5.0
7723,AVpfl8cLLJeJML43AE3S,Easy to set up and works great with hue lighti...,5.0
7724,AVphgVaX1cnluZ0-DR74,Purchased for 10 and 8 yr old for xmas. They l...,5.0
7725,AVphgVaX1cnluZ0-DR74,I purchased this on sale for about $38.Worth e...,5.0


RandomForestClassifier()


In [7]:
#lower case all text
test_data["reviews.text"]=test_data["reviews.text"].str.lower() 

#tokenization of words
test_data['reviews.text'] = test_data.apply(lambda row: word_tokenize(row['reviews.text']), axis=1) 

#only alphanumerical values
test_data["reviews.text"] = test_data['reviews.text'].apply(lambda x: [item for item in x if item.isalpha()]) 

#lemmatazing words
test_data['reviews.text'] = test_data['reviews.text'].apply(lambda x : [WordNetLemmatizer().lemmatize(y) for y in x])

#removing useless words
stop = stopwords.words('english')
test_data['reviews.text'] = test_data['reviews.text'].apply(lambda x: [item for item in x if item not in stop])

In [8]:
test_data["reviews.text"] = test_data["reviews.text"].apply(lambda x: str(' '.join(x))) #joining all tokens

In [9]:
sentiment = {1: 0,
            2: 0,
            3: 0,
            4: 1,
            5: 1}

test_data["sentiment"] = test_data["reviews.rating"].map(sentiment)

In [20]:
# building tfidf matrix to train models 
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer =TfidfVectorizer(max_df=0.9)
text = vectorizer.fit_transform(test_data["reviews.text"])
text

<7727x5779 sparse matrix of type '<class 'numpy.float64'>'
	with 111181 stored elements in Compressed Sparse Row format>

In [62]:


X_tf1 = trained_tfidf_vector.transform(final["reviews.text"])

In [35]:
X_tf1

<7727x10872 sparse matrix of type '<class 'numpy.float64'>'
	with 108805 stored elements in Compressed Sparse Row format>

In [65]:
predicted_result = trained_model_rf.predict(X_tf1)
predicted_proba = trained_model_rf.predict_proba(X_tf1)
predicted_result

array([1, 1, 1, ..., 0, 1, 0], dtype=int64)

0       0.80
1       0.95
2       0.93
3       0.84
4       0.99
        ... 
7722    0.84
7723    0.31
7724    0.24
7725    0.76
7726    0.24
Name: 1, Length: 7727, dtype: float64

In [60]:
result_df = test_data[['reviews.text']]
result_df['actual_rating'] = test_data['reviews.rating']
result_df['Prediction'] = predicted_result
result_df['Probability']=np.round(pd.DataFrame(predicted_proba)[1],2)
result_df

<ipython-input-60-48d6417b0181>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['actual_rating'] = final['reviews.rating']
<ipython-input-60-48d6417b0181>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Prediction'] = predicted_result


,reviews.text,actual_rating,Prediction,Probability
0,decided ditch cable research decided buy insta...,5,1,0.80
1,bought playstation vue new samsung tv,5,1,0.95
2,bought father tv happy easily navigate through...,4,1,0.93
3,device good people trying reduce cable bill,5,1,0.84
4,great streaming plex tv hassle perfect quality...,5,1,0.99
...,...,...,...,...
7722,battery trouble holding charge bought fire jul...,2,1,0.84
7723,daughter ha tablet almost month work well,3,0,0.31
7724,cheap wa impressed never,1,0,0.24
7725,hard use lot ad randomly close apps,2,1,0.76
